In [49]:
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd

## Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.
Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>


In [169]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
KEYWORDS = ['python', 'парсинг']
posts = soup.find_all('article', class_="post post_preview")

# posts[0]

In [170]:
def parse_post_preview(post):
    post_time = re.findall(r'\d{2}:\d{2}', post.find('span', class_='post__time').text)[0]
    title_element = post.find('a', class_='post__title_link')
    post_title = title_element.text
    post_ref = title_element.attrs.get('href')
    return post_time, post_title, post_ref
# parse_post_preview(posts[0], True)

In [171]:
def parse_post_text(post_ref):
    req_text = requests.get(post_ref)
    soup_text = BeautifulSoup(req_text.text, 'html.parser')
    return soup_text.find('div', class_="post__text post__text-html post__text_v1").text

# re.split('; |, |: \*|\s|-',
#          parse_post_text('https://habr.com/ru/company/ruvds/blog/515678/').lower())

In [172]:
KEYWORDS = ['python', 'парсинг', 'вычисления']
result = pd.DataFrame()
for post in posts:
    time.sleep(0.33)
    post_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_time, post_title, post_ref = parse_post_preview(post)
    post_list = re.split('; |, | (|) |: \*|\s|-', post.text.lower())
    if set(KEYWORDS) & set(post_list):
        result = result.append([[post_time, post_title, post_ref]])
        
result.columns=['дата', 'заголовок', 'ссылка']        
result

,дата,заголовок,ссылка
0,20:21,Большая история аргументов функций в Python,https://habr.com/ru/company/ruvds/blog/515678/
0,18:41,Немного об ускорении программы: распараллелива...,https://habr.com/ru/post/515672/
0,16:49,Интернет автомобилей: кому нужны данные о ваше...,https://habr.com/ru/company/itelma/blog/515274/


In [174]:
KEYWORDS = ['python', 'java', 'linux', 'парсинг', 'вычисления']
result = pd.DataFrame()
for post in posts:
    time.sleep(0.33)
    post_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_time, post_title, post_ref = parse_post_preview(post)
    post_list = re.split('; |, | (|) |: \*|\s|-|\n', post.text.lower())
    parse_text = parse_post_text(post_ref)
    parse_text_splited = re.split('; |, | (|) |: \*|\s|-|\n', parse_text.lower())
    post_list.extend(parse_text_splited)
    if set(KEYWORDS) & set(post_list):
        result = result.append([[post_time, post_title, post_ref, parse_text]])

result.columns=['дата', 'заголовок', 'ссылка', 'текст_статьи']
result

,дата,заголовок,ссылка,текст_статьи
0,20:21,Большая история аргументов функций в Python,https://habr.com/ru/company/ruvds/blog/515678/,"Ну, на самом деле, история аргументов в Python..."
0,19:07,Java-апплеты будут исключены из Java 8 с оконч...,https://habr.com/ru/company/rdtex/blog/515680/,"Хорошо известна шутливая истина, что любая дей..."
0,18:41,Немного об ускорении программы: распараллелива...,https://habr.com/ru/post/515672/,"Здравствуйте, уважаемые читатели. В этой публи..."
0,17:57,"«Наша школа — это больше, чем просто источник ...",https://habr.com/ru/company/tsystems/blog/515644/,"Один из вечных вопросов, которыми задаются рук..."
0,17:32,Восторг безопасника — технология для шифровани...,https://habr.com/ru/company/rebrainme/blog/515...,На днях поступила интересная задачка — необход...
0,17:25,Искусственный интеллект в области юриспруденци...,https://habr.com/ru/post/515510/,Введение\nМы продолжаем цикл статей на тему юр...
0,16:49,Интернет автомобилей: кому нужны данные о ваше...,https://habr.com/ru/company/itelma/blog/515274/,\n\nРаспределенные вычисления в локальной сети...
0,16:30,Что такое Docker: краткий экскурс в историю и ...,https://habr.com/ru/company/southbridge/blog/5...,10 августа в Слёрм стартовал видеокурс по Dock...


## Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.
Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [145]:
EMAIL = ['xxx@x.ru', 'yyy@y.com', 'my@gmail.com', 'mymail@mail.ru']

In [175]:
def hack_check(EMAIL: list = None):
    if EMAIL is None:
        EMAIL = ['xxx@x.ru', 'yyy@y.com', 'my@gmail.com', 'mymail@mail.ru']
    df = pd.DataFrame()
    for mail in EMAIL:
        req = requests.post('https://digibody.avast.com/v1/web/leaks/',
                            json={'email': mail})
        dat = pd.json_normalize(req.json()['value'])[[
                                                'leak_date',
                                                'leak_info.title',
                                                'leak_info.description',
                                               ]]
        dat.insert(0, 'mail', mail)
        df = df.append(dat)
        time.sleep(0.33)
    df.leak_date = pd.to_datetime(df.leak_date, unit='ms', origin='unix')
    df.columns=['почта', 'дата утечки', 'источник утечки', 'описание утечки']
    return df

hack_check().tail()

,почта,дата утечки,источник утечки,описание утечки
38,mymail@mail.ru,2019-01-25,Collection #1 Combo List,"On January 7, 2019, an online user named Sanix..."
39,mymail@mail.ru,2016-11-01,Qip,"In 2011, Russian instant messaging service pro..."
40,mymail@mail.ru,2019-12-05,Sensitive Source,This source has been marked as sensitive due t...
41,mymail@mail.ru,2017-05-26,Leet.cc,"In February 2016, LEET's user database was all..."
42,mymail@mail.ru,2020-08-13,"""AntiPub Drugs Squad"" Combolist","In August 2020, this combolist was shared on a..."


In [186]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = '7'
VERSION = '5.103'
SLEEP = 0.33
GROUP = 'netology'
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50,
    'domain':GROUP,
}
res = requests.get(NEWSFEED_REQUEST, params)
df = pd.DataFrame(res.json()['response']['items'])[['date', 'text']]
df.date = pd.to_datetime(df.date, unit='s')
df

,date,text
0,2020-08-14 11:00:08,Всем привет! Сегодня первый день карьерного кв...
1,2020-08-16 12:28:00,"Подборка книг, которые помогут развить математ..."
2,2020-08-16 08:30:24,Продвижение курса — дело непростое. Но у вас к...
3,2020-08-15 11:42:00,Личный бренд работает по принципу зачётки: сна...
4,2020-08-15 08:30:17,"Как успехи у ваших студентов? Поздравляем, вы ..."
5,2020-08-15 07:42:00,*партнерский пост*\n\nМеждународный конкурс по...
6,2020-08-14 11:00:12,А вот и первое задание карьерного квеста. Мене...
7,2020-08-14 07:42:00,"Больше, ещё больше знаний🤓 Разбираемся с дизай..."
8,2020-08-13 15:14:00,"«Нужно работать так, словно таланта у тебя воо..."
9,2020-08-13 07:42:00,"Составляем карту пути клиента, открываем калит..."
